In [91]:
pronoun = "oни"

vocab = ["oни", "вас",  "вы", "их","меня", "мне", "мы","нас", "тебя","ты", "я"]

In [92]:
from pathlib import Path

all_txt_files =[]
for file in Path("../txts/Pronoun/"+pronoun).rglob("*.txt"):
     all_txt_files.append(file.parent / file.name)
# counts the length of the list
n_files = len(all_txt_files)

In [93]:
all_txt_files.sort()
all_txt_files[0]

WindowsPath('../txts/Pronoun/oни/2020-03-01-pre-они-pronoun.txt')

In [94]:
all_docs = []
for txt_file in all_txt_files:
    with open(txt_file, encoding='utf-8') as f:
        txt_file_as_string = f.read()
    all_docs.append(txt_file_as_string)

In [95]:
#import the TfidfVectorizer from Scikit-Learn.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=.65, min_df=1, stop_words=None, use_idf=True, norm=None)
transformed_documents = vectorizer.fit_transform(all_docs)

In [96]:
transformed_documents_as_array = transformed_documents.toarray()
# use this line of code to verify that the numpy array represents the same number of documents that we have in the file list
len(transformed_documents_as_array)

122

In [97]:
print(transformed_documents_as_array)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [98]:
import pandas as pd

# make the output folder if it doesn't already exist
Path("../tf_idf_output").mkdir(parents=True, exist_ok=True)

# construct a list of output file paths using the previous list of text files the relative path for tf_idf_output
output_filenames = [str(txt_file).replace(".txt", ".csv").replace("txt/", "tf_idf_output/") for txt_file in all_txt_files]

all_list = []

# loop each item in transformed_documents_as_array, using enumerate to keep track of the current position
for counter, doc in enumerate(transformed_documents_as_array):
    # construct a dataframe
    tf_idf_tuples = list(zip(vectorizer.get_feature_names(), doc))
    one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['TERM', 'SCORE']).sort_values(by='SCORE', ascending=False).reset_index(drop=True).head(20)
    all_list.append(tf_idf_tuples)
    # output to a csv using the enumerated value for the filename
    # one_doc_as_df.to_csv(output_filenames[counter])

In [99]:
flat = []


for i in all_list:
    for j in i:
        flat.append(j)

all_doc_as_df = pd.DataFrame.from_records(flat, columns=['TERM', 'SCORE']).sort_values(by='SCORE', ascending=False).reset_index(drop=True).head(20)
all_doc_as_df.to_csv('../tf_idf_output/'+pronoun+'.csv', encoding='utf-8-sig')